In [1]:
# Instala la librería pycryptodome
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 327.2 kB/s eta 0:00:00a 0:00:06

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# **AES-CBC**

El siguiente código escrito en python cifra/descifra un archivo.


*   El método encrypt(key, filename) recibe una llave (key), que puede ser generada por getKey(password), y la extensión del archivo a cifrar (filename).
*   El método decrypt(key, filename) recibe una llave (key), que puede ser generada por getKey(password), y la extensión del archivo a cifrar (filename).



In [6]:
import os
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Hash import SHA256
from Crypto.Util.Padding import pad, unpad

# Función para cifrar un archivo
def encrypt(key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # Nombre del archivo de salida cifrado
        outputFile = filename+".enc"
        # Obtener el tamaño del archivo original y rellenar con ceros a la izquierda para que tenga 16 bytes
        filesize = str(os.path.getsize(filename)).zfill(16)
        # Generar un vector de inicialización (IV) aleatorio de 16 bytes
        IV = Random.new().read(16)
        # Crear un objeto cifrador AES en modo CBCvsECB (Cipher Block Chaining)
        encryptor = AES.new(key, AES.MODE_CBC, IV)

        # Abrir el archivo de entrada en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Escribir el tamaño del archivo original codificado en utf-8
                        outfile.write(filesize.encode('utf-8'))
                        # Escribir el vector de inicialización (IV)
                        outfile.write(IV)

                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break
                                # Si el tamaño del chunk no es múltiplo de 16, rellenar con bytes nulos
                                elif len(chunk)%16 != 0:
                                        chunk=pad(chunk, 16)

                                # Cifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(encryptor.encrypt(chunk))

# Función para descifrar un archivo
def decrypt(key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # ruta del archivo de salida descifrado (eliminar la extensión .enc)
        outputFile = filename[:-4]

        # Abrir el archivo de entrada cifrado en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Leer el tamaño del archivo original (los primeros 16 bytes) y convertirlo a entero
                filesize = int(infile.read(16))
                # Leer el vector de inicialización (IV) (los siguientes 16 bytes)
                IV = infile.read(16)

                # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
                decryptor= AES.new(key, AES.MODE_CBC, IV)

                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break

                                # Descifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(decryptor.decrypt(chunk))

                        # Truncar el archivo de salida al tamaño original
                        outfile.truncate(filesize)

# Función para generar una clave a partir de una contraseña usando SHA256
def getKey(password):
        # Crear un objeto hash SHA256
        hasher = SHA256.new(password.encode('utf-8'))
        # Devolver el resumen del hash (la clave)
        return hasher.digest()

# Función principal
def Main():
        # Pedir al usuario si desea cifrar o descifrar
        choice = input("Te gustaría Cifrar (C) o descifrar (D)")

        # Si la opción es C, cifrar el archivo
        if choice == 'C':
                filename = input("Ruta de archivo a cifrar : ")
                password = input("Contraseña: ")
                encrypt(getKey(password), filename)
                print('Hecho.')
        # Si la opción es D, descifrar el archivo
        elif choice == 'D':
                filename = input("Ruta de archivo a decifrar : ")
                password = input("Contraseña: ")
                decrypt(getKey(password),filename)
                print("Hecho.")

        # Si la opción no es válida, mostrar un mensaje
        else:
                print("Ninguna opción ha sido seleccionada.")

# Ejecutar la función principal
Main()
#/content/drive/MyDrive/ejemplo_cifrado_AES/

Hecho.


## Bit FLipper

In [1]:
with open ("test.pdf.enc", "rb") as f :
    data = bytearray(f.read())
    data [50] ^= 0x01

# cambia un bit dentro de C_3 ( ciphertext )
with open ("archivo_mod.enc", "wb") as f :
    f.write(data)

## Block Swaper

In [48]:
def swap_blocks(input_file, output_file, block_size=16, i=2, j=3):
    """
    Intercambia los bloques Ci y Cj de un archivo cifrado en formato:
    C0 || C1 || C2 ... Cn
    block_size normalmente = 16 (AES).
    """

    with open(input_file, "rb") as f:
        data = bytearray(f.read())

    #print(data)
    # Calcular offset en bytes de cada bloque
    start_i, end_i = i * block_size, (i+1) * block_size
    start_j, end_j = j * block_size, (j+1) * block_size

    # Intercambiar bloques
    block_i = data[start_i:end_i]
    block_j = data[start_j:end_j]
    data[start_i:end_i], data[start_j:end_j] = block_j, block_i

    with open(output_file, "wb") as f:
        f.write(data)

    print(f"Intercambiados C{i} y C{j} en {input_file}, guardado en {output_file}")


## Comparador de files bit a bit

In [56]:
def compare_files_per_byte(file1, file2, block_size=16, max_diffs=64):
    """
    Compara dos archivos byte por byte y muestra diferencias.
    Agrupa por bloques para ver qué zonas se dañaron.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()

    for i, (b1, b2) in enumerate(zip(data1, data2)):
        if b1 != b2:
            block_index = i // block_size
            print(f"Diferencia en byte {i} (bloque {block_index}): {b1} != {b2}")
            max_diffs -= 1
            if max_diffs == 0:
                print("...más diferencias ocultas...")
                break



In [57]:
def compare_files_per_block(file1, file2, block_size=16):
    """
    Compara dos archivos y reporta únicamente los bloques que están corrompidos.
    """
    with open(file1, "rb") as f:
        data1 = f.read()
    with open(file2, "rb") as f:
        data2 = f.read()



    corrupted_blocks = set()

    # Comparar byte por byte y registrar bloques corrompidos
    for i in range(min(len(data1), len(data2))):
        if data1[i] != data2[i]:
            block_index = i // block_size
            corrupted_blocks.add(block_index)

    # Mostrar resultados
    if corrupted_blocks:
        sorted_blocks = sorted(corrupted_blocks)
        print(f"Bloques corrompidos: {sorted_blocks}")
        print(f"Total de bloques afectados: {len(corrupted_blocks)}")
    else:
        print("Los archivos son idénticos - no hay bloques corrompidos")

# **AES-ECB**

In [8]:
def encrypt_ecb(key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # Nombre del archivo de salida cifrado
        outputFile = filename+".enc"
        # Obtener el tamaño del archivo original y rellenar con ceros a la izquierda para que tenga 16 bytes
        filesize = str(os.path.getsize(filename)).zfill(16)
        # Generar un vector de inicialización (IV) aleatorio de 16 bytes

        #IV = Random.new().read(16)

        # Crear un objeto cifrador AES en modo ECB (Electronic Code Book)
        encryptor = AES.new(key, AES.MODE_ECB)

        # Abrir el archivo de entrada en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Escribir el tamaño del archivo original codificado en utf-8
                        outfile.write(filesize.encode('utf-8'))


                        # Escribir el vector de inicialización (IV)
                        #outfile.write(IV)


                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break
                                # Si el tamaño del chunk no es múltiplo de 16, rellenar con bytes nulos
                                elif len(chunk)%16 != 0:
                                        chunk=pad(chunk, 16)

                                # Cifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(encryptor.encrypt(chunk))

# Función para descifrar un archivo
def decrypt_ecb(key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # ruta del archivo de salida descifrado (eliminar la extensión .enc)
        outputFile = filename[:-4]

        # Abrir el archivo de entrada cifrado en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Leer el tamaño del archivo original (los primeros 16 bytes) y convertirlo a entero
                filesize = int(infile.read(16))


                # Leer el vector de inicialización (IV) (los siguientes 16 bytes)
                #IV = infile.read(16)

                # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
                decryptor= AES.new(key, AES.MODE_ECB)

                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break

                                # Descifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(decryptor.decrypt(chunk))

                        # Truncar el archivo de salida al tamaño original
                        outfile.truncate(filesize)

In [62]:
def encrypt(mode, key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # Nombre del archivo de salida cifrado
        outputFile = filename+".enc"
        # Obtener el tamaño del archivo original y rellenar con ceros a la izquierda para que tenga 16 bytes
        filesize = str(os.path.getsize(filename)).zfill(16)
        # Generar un vector de inicialización (IV) aleatorio de 16 bytes

        # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
        if mode == "ECB":
            encryptor= AES.new(key, AES.MODE_ECB)
        elif mode == "CBC":
            IV = Random.new().read(16)
            encryptor= AES.new(key, AES.MODE_CBC, IV)
        else:
            print("Error de mode")
            return

        # Abrir el archivo de entrada en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Escribir el tamaño del archivo original codificado en utf-8
                        outfile.write(filesize.encode('utf-8'))


                        # Escribir el vector de inicialización (IV)
                        #outfile.write(IV)


                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break
                                # Si el tamaño del chunk no es múltiplo de 16, rellenar con bytes nulos
                                elif len(chunk)%16 != 0:
                                        chunk=pad(chunk, 16)

                                # Cifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(encryptor.encrypt(chunk))

# Función para descifrar un archivo
def decrypt(mode, key, filename):
        # Tamaño del chunk para leer el archivo
        chunksize = 64*1024
        # ruta del archivo de salida descifrado (eliminar la extensión .enc)
        outputFile = filename[:-4]

        # Abrir el archivo de entrada cifrado en modo binario de lectura ('rb')
        with open(filename, 'rb') as infile:
                # Leer el tamaño del archivo original (los primeros 16 bytes) y convertirlo a entero
                filesize = int(infile.read(16))

                # Crear un objeto descifrador AES en modo CBCvsECB (Cipher Block Chaining)
                if mode == "ECB":
                    decryptor= AES.new(key, AES.MODE_ECB)
                elif mode == "CBC":
                    IV = infile.read(16)
                    decryptor= AES.new(key, AES.MODE_CBC, IV)
                else:
                    print("Error de mode")
                    return

                # Abrir el archivo de salida en modo binario de escritura ('wb')
                with open(outputFile, 'wb') as outfile:
                        # Leer el archivo en chunks
                        while True:
                                chunk = infile.read(chunksize)

                                # Si no hay más chunks, salir del bucle
                                if len(chunk) == 0:
                                        break

                                # Descifrar el chunk y escribirlo en el archivo de salida
                                outfile.write(decryptor.decrypt(chunk))

                        # Truncar el archivo de salida al tamaño original
                        outfile.truncate(filesize)

## Parte 3 - Intercambio de Bloques

In [63]:
encrypt("CBC", getKey("1234"), "test.pdf")
swap_blocks("test.pdf.enc", "Resultados/CBC - Block Swap/test_swap.pdf.enc", 16, 3, 5)
decrypt("CBC", getKey("1234"), "Resultados/CBC - Block Swap/test_swap.pdf.enc")
compare_files_per_byte("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")
print("=====================================================================================================================")
compare_files_per_block("test.pdf", "Resultados/CBC - Block Swap/test_swap.pdf")

Intercambiados C3 y C5 en test.pdf.enc, guardado en Resultados/CBC - Block Swap/test_swap.pdf.enc
Diferencia en byte 0 (bloque 0): 37 != 32
Diferencia en byte 1 (bloque 0): 80 != 48
Diferencia en byte 2 (bloque 0): 68 != 32
Diferencia en byte 3 (bloque 0): 70 != 111
Diferencia en byte 4 (bloque 0): 45 != 98
Diferencia en byte 5 (bloque 0): 49 != 106
Diferencia en byte 6 (bloque 0): 46 != 10
Diferencia en byte 7 (bloque 0): 53 != 60
Diferencia en byte 8 (bloque 0): 10 != 60
Diferencia en byte 9 (bloque 0): 37 != 10
Diferencia en byte 10 (bloque 0): 208 != 47
Diferencia en byte 11 (bloque 0): 212 != 76
Diferencia en byte 12 (bloque 0): 197 != 101
Diferencia en byte 13 (bloque 0): 216 != 110
Diferencia en byte 14 (bloque 0): 10 != 103
Diferencia en byte 15 (bloque 0): 54 != 116
Diferencia en byte 16 (bloque 1): 32 != 230
Diferencia en byte 17 (bloque 1): 48 != 176
Diferencia en byte 18 (bloque 1): 32 != 29
Diferencia en byte 19 (bloque 1): 111 != 98
Diferencia en byte 20 (bloque 1): 98 !=

## Pruebas en ECB